In [98]:
import requests
import pandas as pd
import re
from geographiclib.geodesic import Geodesic
import warnings
warnings.filterwarnings("ignore")

In [111]:
LAT_A = 52.162258
LNG_A = 21.028076
LAT_B = 52.1624
LNG_B = 21.0383

In [ ]:
def fetch_nextbike_data() :
    url = "https://api.nextbike.net/maps/nextbike-live.json?city=199,362,812,833"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        return None
    except:
        return None

In [ ]:
def karney_distance(lat1, lon1, lat2, lon2):
    geod = Geodesic.WGS84
    result = geod.Inverse(lat1, lon1, lat2, lon2)
    return result['s12'] / 1000  # Distance in kilometers

In [90]:
def filter_stations(stations):
    # filter stations with name like BIKE 612448, use regex
    stations = [station for station in stations if not re.match(r"^BIKE \d+$", station["name"])]
    return stations

In [34]:
def sort_stations(lat, lng, places):
    # add distance to each place
    for place in places:
        place['distance'] = karney_distance(lat, lng, place['lat'], place['lng'])
    # sort by distance
    places.sort(key=lambda x: x['distance'])
    return places

In [ ]:
def find_connection(df_history, lat_a, lng_a, lat_b, lng_b):
    data = fetch_nextbike_data()
    places = data["countries"][2]["cities"][0]["places"]
    places = filter_stations(places)
    places = sort_stations(lat_a, lng_a, places)
    for place in places:
        bikes = place.get("bike_list", [])
        if len(bikes) > 0:
            break
    start_place = place.copy()

    places = sort_stations(lat_b, lng_b, places)
    for place in places:
        if place["free_racks"] > 0:
            break
    end_place = place.copy()

    if start_place == end_place:
        print("Start and end places are the same.") # if both stations are the same, no good bike connection avialable
        return None, None

    slct = df_history[
        (df_history["from_station"] == start_place["name"]) &
        (df_history["to_station"] == end_place["name"])
    ]
    slct["departure_time"] = pd.to_datetime(slct["departure_time"])
    slct["arrival_time"] = pd.to_datetime(slct["arrival_time"])
    slct["travel_time"] = slct["arrival_time"] - slct["departure_time"]

    print(f"Closest station with bikes: {start_place['name']}, {start_place['distance']:.2f} km away, {len(bikes)} bikes available.")
    print(f"Closest station with free racks: {end_place['name']}, {end_place['distance']:.2f} km away, {end_place['free_racks']} free racks available.")

    if len(slct) > 0:
        print(f"Previous trips from {start_place['name']} to {end_place['name']}:")
        print(f"Number of trips: {len(slct)}")
        print(f"Average travel time: {slct['travel_time'].mean().total_seconds() / 60:.2f} minutes.")
        print(f"Maximum travel time: {slct['travel_time'].max().total_seconds() / 60:.2f} minutes.")
        print(f"Minimum travel time: {slct['travel_time'].min().total_seconds() / 60:.2f} minutes.")
    else:
        print(f"No previous trips from {start_place['name']} to {end_place['name']}.")

    return start_place, end_place

In [ ]:
df = pd.read_csv("bike_movements_warsaw.csv")

In [113]:
a, b = find_connection(df, LAT_A, LNG_A, LAT_B, LNG_B)

Closest station with bikes: Metro Ursynów, 0.00 km away, 1 bikes available.
Closest station with free racks: Jastrzębowskiego - SGGW, 0.01 km away, 4 free racks available.
Previous trips from Metro Ursynów to Jastrzębowskiego - SGGW:
Number of trips: 302
Average travel time: 36.85 minutes.
Maximum travel time: 1062.92 minutes.
Minimum travel time: 3.08 minutes.
